In [42]:
import numpy as np
from numpy import random as rd
from pykrx import stock
import pandas as pd
import matplotlib.pyplot as plt

#아래 티커는 각각 kodex레버리지 / KODEX 200  / ARIRANG고배당주
ETF_ticker = ["148070"]
N = len(ETF_ticker) #ETF 갯수만큼 자산 총 갯수는 N개로 제한된다.
stocks_data = []
srt = "20120101"
end = "20191231"
for i in range(0, N):
    stocks_data.append(stock.get_market_ohlcv_by_date(srt, end, ETF_ticker[i])["종가"][-300:])#룩백윈도우는 300일


In [41]:
pd.DataFrame(np.asarray(stocks_data).T)

,0
0,114039
1,114240
2,114560
3,114929
4,114694
...,...
295,123163
296,123363
297,123290
298,123085


In [4]:
ret = pd.DataFrame(np.nan,index=np.arange(len(stocks_data[0])), columns=ETF_ticker)
for stock, idx in zip(ETF_ticker, range(0,N)):
    for i in range(1, len(stocks_data[0])):
        ret[stock].iloc[i] =  stocks_data[idx][i]/stocks_data[idx][i-1] 

In [5]:
retArr = []
for stock in ETF_ticker:
    retArr.append( ret[stock].iloc[1:] )# 0일 수익률은 nan 값이라서 날짜 체크할때 하루 밀리는거 조심할것.

In [18]:
np.cumprod(retArr[0])

1      0.996033
2      1.021023
3      1.001983
4      1.009917
5      1.015073
         ...   
295    1.120587
296    1.105117
297    1.114240
298    1.149147
299    1.144387
Name: 122630, Length: 299, dtype: float64

In [8]:
#var - cov matrix 생성
def getCov(a, b):
    return np.cov(a,b)

var_cov_mat = np.identity(N) # N * N 단위행렬

for i in range(0, N):
    for j in range(0, N):
        var_cov_mat[i][j] = getCov(stocks_data[i], stocks_data[j])[0,1]

In [22]:
a = []
for i in range(20*12):
    a.append(i*100 + 10000)

In [21]:
np.std(a)

6928.143089361439

In [20]:
b = []
for i in range(12):
    b.append(i*1900 + 10000)
np.std(b)

6558.89980611586

In [9]:
#가격이 아닌 수익률에 대한 cov mat 생성하기.
for i in range(0, N):
    for j in range(0, N):
        var_cov_mat[i][j] = getCov(retArr[i], retArr[j])[0,1] * 1000

In [21]:
getCov(retArr[i], retArr[j])

array([[6.22802966e-05, 6.22802966e-05],
       [6.22802966e-05, 6.22802966e-05]])

In [34]:
var_cov_mat

array([[0.31584232, 0.15551405, 0.09884801],
       [0.15551405, 0.07697095, 0.04895697],
       [0.09884801, 0.04895697, 0.0622803 ]])

In [23]:
np.var(retArr[0])

0.0003147859955994781

In [24]:
np.std(retArr)

0.012297967382004601

In [6]:
import scipy as sp
from scipy import optimize

def eq_constraint(w):
    summation = 0
    for i in w:
        summation += i
    return summation -1

def eq_nonnegative(w):
    non_negative = True
    for i in w:
        if i < 0:
            non_negative = False
    if non_negative:
        return 0
    return 0

def flarray(w):
    summation = 0
    risk = np.dot(np.dot(np.transpose(w), var_cov_mat), w) # sigma _ w
    N = len(w)
    for wi, i in zip(w, range(0,N)):
        summation += (wi - risk/(N* np.array(np.dot(var_cov_mat, w)[i])))**2
    return summation

In [10]:
w_init = [1/N for i in range(0,N)] # 초기값은 동일비중[1/4 1/4 1/4 1/4]
result1 = sp.optimize.fmin_slsqp(flarray, w_init, eqcons=[eq_constraint])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2.575727968343243e-07
            Iterations: 5
            Function evaluations: 26
            Gradient evaluations: 5


In [12]:
result1

array([0.18135365, 0.36730141, 0.45134494])

In [28]:
#초기값을 달리했을때
w_another_init = [0.4, 0.3, 0.3]
result2 = sp.optimize.fmin_slsqp(flarray, w_another_init, eqcons=[eq_constraint])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.3498944693059356e-07
            Iterations: 3
            Function evaluations: 17
            Gradient evaluations: 3


In [29]:
print(result1)
print(result2)

[0.18135365 0.36730141 0.45134494]
[0.1817393  0.36740551 0.45085519]


In [30]:
print(result1)
print(result2)
print(result1 - result2)

[0.18135365 0.36730141 0.45134494]
[0.1817393  0.36740551 0.45085519]
[-0.00038565 -0.0001041   0.00048975]


In [33]:
w_extreme = [0.4 , 0.1, 0.3, 0.2]
sp.optimize.fmin_slsqp(flarray, w_extreme, eqcons=[eq_constraint])

ValueError: shapes (4,) and (3,3) not aligned: 4 (dim 0) != 3 (dim 0)